### AVALIAÇÃO FINAL   
### MÓDULO EXPLORAÇÃO DE DADOS NA INTERNET

### Objetivo
####Raspar os dados de uma livraria online: http://books.toscrape.com/
#### Extrair apenas duas informações de cada livro: 
 1. Título 
 2. Descrição.

####Bibliotecas

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

####Função para solicitar e analisar a página Web

In [ ]:
url = "http://books.toscrape.com/index.html"

def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

print(getAndParseURL(url).prettify()[:1000])

####Função para buscar links de livros em qualquer página do site

In [3]:
def getBooksURLs(url):
    soup = getAndParseURL(url)
    
    return(["/".join(url.split("/")[:-1]) + "/" + x.div.a.get('href')
       for x in soup.findAll("article", class_ = "product_pod")])

####Obter todas as páginas URLs

In [ ]:
pages_urls = [url]

soup = getAndParseURL(pages_urls[0])

while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(pages_urls) == 1:
    new_url = "/".join(pages_urls[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
    pages_urls.append(new_url)
    soup = getAndParseURL(new_url)
    
print(str(len(pages_urls)) + " fetched URLs")
print("Some examples:")
pages_urls[:5]

####Busca todos os URLs dos produtos para cada página

In [ ]:
booksURLs = []

for page in pages_urls:
    booksURLs.extend(getBooksURLs(page))
    
print(str(len(booksURLs)) + " fetched URLs")
print("Some examples:")
booksURLs[:5]

####Busca os dados de cada produto

In [6]:
title = []
description = []

for url in booksURLs:
    soup = getAndParseURL(url)
    # product title
    title.append(soup.find("div", class_ = re.compile("product_main")).h1.text)
    # product description
    description.append(soup.find_all('p')[3].get_text())

####Armazena dentro de um DataFrame do Pandas.

In [ ]:
df_scraped = pd.DataFrame({'Title': title, 'Description': description})

df_scraped